# GPy Hack Day

To demonstrate the paramz module we're going to consider the simple ridge-regression model.

"We'll try to fit a function $f(x)$ with polynomial basis functions. Kernel ridge regression does this by forming an objective function that is a combination of a l2 loss and a penalty that is there oto avoid the weights going too large"

$\sum_{i=1}^N (y-\phi(x_t)^\top w)^2 + \lambda \sum_{j=0}^m w_j^2$

We'll have a 2nd order polynomial so number of features $m$ goes from 0 to 2.

## What are the parameters in paramz?

- They are **not probabilistic** (note; in GPy you can add priors, but they are still fixed determined values). Instead they are fitted, and are then fixed - i.e. they're not integrated over. (I.e. are hyperparameters).

    TODO: In Alan's notebook we should pull code of model to notebook. -> DONE in Max's notebook?

### Why might checkgrad fail?
    
There are two things that could be wrong, if checkgrad fails;

 - the objective function could be wrong
 - the gradients could be wrong
 
Also you might have to adjust the step size and ensure you're not at the mode (or a maximum). 
 
Note: In paramz there is no log-likelihood, as we're not doing anything probabilistic.

### Constraining things

e.g.

    m.weights.constrain_fixed(value=1.0)

**!Tying parameters doesn't work at the moment**

### How does paramz handle boundary constraints?

- It uses a sigmoid
- One issue is that it will take a long time to get to solutions very close to the boundary, as your gradient will approach zero as you get close to the boundary.

    TODO: Suggest a tie kernel should be available! E.g. https://github.com/lionfish0/clustering/blob/master/Example%20using%20the%20Tie%20Kernel.ipynb

## Paramz Framework

You always inherit from a model, e.g.;

    class RidgeRegressionSimple(Model):
    
Where Model is in the paramz module. We'll cover more complex models later.

### Constructor

Note: We save our values as observable arrays (ObsAr) - these are like numpy arrays but lets us cache results of functions, e.g. if you call a method with that array twice, the second time will be quick.

The key part re paramz is:

    weights = Param('weights', weights)
    self.weights = weights
    self.link_parameter(weights)
    
this means it knows it has gradients, constraints and where it sits in the model. Note that you can choose its name here.

Side note:
> Later you can change the name,
>
>    m.weights.name = 'weights_for_ridge'
>
> It's best to provide unique names, as it'll add extra numbers onto the names to make them unique otherwise.

Another side note:

> The Cacher import - lets us add a decorator which allows us to cache the results of functions;
>
>   @Cache_this
>   def myfunction(X):
>       ...
    
### Param vs Parameterized

Nodes are `parameterized`, leaves are Param

Anything that should contain parameters is inherited from `Parameterized`, anything that is a parameter is inherited from `Param`. To make the link we call the `link_parameter` with the variable.

For example we might have a GP model (as the top level). This might have a parameter (such as the noise variance):

Parameterized [GP Model]
    - Parameterized [kernel]
        - Param [lengthscale]
        - Param [variance]
    - Param [noise variance]
    - Parameterized [another kernel?]
    
Comment: !Bit slow in deep hierarchical models; should update in different orders.

When you change, e.g the lengthscale of the kernel, it'll effectively need to update the kernel and that will trigger an update of the GP Model - as this will need to know that the likelihood has to be recomputed.

The Param class is inherited from numpy array, so it can be treated just as a numpy array.

### parameters_changed method:

Any parameters that you've linked (e.g. the weight in the regression model) the model will automatically call 'parameters_changed') - this means that we can update things like the objective, the gradients, etc.

Note: The model actually calls the method `objective_function()` etc when it's optimising etc. but it's worth computing such stuff in `parameters_changed` so we can then just return it when required.

### .gradient

Any parameter has an attribute called `gradient` which we set here; e.g.
    
    self.weight.gradient[:] = self._lambda*2*self.weights
    
E.g. If you thought that the weights had different gradients, then you could do

    self.weight[2].gradient[:] = self._lambda*2*self.weights
    
The `[:]` is so that we set the content of the gradient array (not try to point it somewhere else).

### Summary

Every time the gradient changes the `parameters_changed` gets called... which updates the gradients and objectives. We do this repeatedly until we reach the mode of the objective.

## GPy

So far we've just looked at a simple `Paramz` model.

Let's look at GPy.

A GPy model is structured as follows;

```
                     Model
               /    |         \
              /     |           \
             /      |             \
 inference_method  likelihood      kernel
                    |               /     \
             noise_variance  lengthscale variance
```

The likelihood and the kernel are likly to have their own parameters (e.g. noise variane in the likelihood function and the kernel could have lengthscale, etc).

#### Inference_method

The objective function is the negative log marginal likelihood:

$log p(y|\sigma_l^2, \theta_k) = \int p(y|f, \sigma_l^2) p(f|\theta_k) df$

where there's a latent function that **is** integrated over, and some hyperparameters that are not.

The two distributions (the prior and the data-likelihood) are both normal, so the marginal likelihood (proportional to the posterior) is also normal;

$log\;\;N(y|0, K(X,X) + \sigma^2 I)$

We can differentiate this log likelihood with respect to the two (or more) (hyper)parameters.

#### Likelihood method

If the likelihood is not Gaussian then can't do the simple analytical solution above, e.g. it could be the bernoulli distribution. So we have to approximate the result.

When you change the likelihood the inference method must be changed.

### Visiting the code... gp.py

Things to note:

- GPy's gp.py does normalising for you [TODO maybe this needs clarifying somewhere?]
- Y_metadata, useful for e.g. multiple output GPs, so you know which output is which

The kernel and likelihood are objects inherited from parameterized (i.e. they contain parameters)

inference_method is just an object.

In [1]:
import GPy
import numpy as np
X = np.arange(0,10,0.2)[:,None]
Y = np.sin(X)+np.random.randn(len(X))

kern = GPy.kern.RBF(1)
lik = GPy.likelihoods.Gaussian(variance=1e-2)
exact = GPy.inference.latent_function_inference.ExactGaussianInference()
exact = GPy.inference.latent_function_inference.ExactGaussianInference()
GPy.core.GP(X=X, Y=Y, kernel=kern, likelihood=lik, inference_method=exact)

warning in stationary: failed to import cython module: falling back to numpy


gp.,value,constraints,priors
rbf.variance,1.0,+ve,
rbf.lengthscale,1.0,+ve,
Gaussian_noise.variance,0.01,+ve,


[Bit vague on this;]

Note: Inside gp.py we link the kernel and likelihood, so when the kernel changes, this propagates to the model, which updates things like the objective... etc.

All the inference methods take, X, the likelihood method and the kernel, and requires [TODO: What?]

The parameters_changed method is implemented, and calls the `inference` method of the inference_method we chose earlier.
This method also calls the kernel's `update_gradients_full`.

To find out more about this and writing new kernels, see http://pythonhosted.org/GPy/tuto_creating_new_kernels.html

# Writing kernels, notes

## Implementing kernels

I realise all we covered in the talk is at http://pythonhosted.org/GPy/tuto_creating_new_kernels.html

## Kernel slicing

Will probably be in the kernel documentation somewhere,

`k = GPy.kern.RBF(1,active_dims=[0]) + GPy.kern.Matern52(1,active_dims=[1])`

# paramz example

This is Max's paramz example details; could probably be moved out to a new notebook

This notebook demonstrates optimising a model with paramz. GPy uses paramz to optimise hyperparameters.

In [3]:
import paramz
import numpy as np

#we'll use the rosenbrock function to demonstrate this - we want to minimise the rosenbrock function
from scipy.optimize import rosen_der, rosen

#start point
x = np.array([4,1])

In [4]:
#We make our model "Rosen", that inherits Model so that it includes
#Parameterized which allows us to have a parameter that's optimised.
class Rosen(paramz.Model):
    
    #the constructor calls the parent constructor, and links the one parameter to the model.
    def __init__(self, x, name='rosen'):
        super(Rosen,self).__init__(name=name)
        self.x = paramz.Param('position',x)
        self.link_parameter(self.x)
        
    #this gives the objective function, note that usually we'd store the objective
    #in an instance variable, e.g. in parameters_changed (to cache it) for use later.
    def objective_function(self):
        return rosen(self.x)
    
    #this is run every time a parameter is altered.
    def parameters_changed(self):
        self.x.gradient = rosen_der(self.x)

#create the object
r = Rosen(x)

We can see that if we change a parameter, the objective is updated automatically;

In [5]:
print(r)
r.x[0]=10
print(r)


Name : rosen
Objective : 22509.0
Number of Parameters : 2
Number of Optimization Parameters : 2
Updates : True
Parameters:
  rosen.    |  value  |  constraints
  position  |   (2,)  |             

Name : rosen
Objective : 980181.0
Number of Parameters : 2
Number of Optimization Parameters : 2
Updates : True
Parameters:
  rosen.    |  value  |  constraints
  position  |   (2,)  |             


Let's optimise and check that we're in the right place [1,1]

In [8]:
r.optimize()

print(r.x)

  index  |  rosen.position  |  constraints
  [0]    |      1.00000000  |             
  [1]    |      1.00000000  |             


### Constraints

Easy to add;

e.g.
    `r.x.constrain_bounded(-2.0,0.5)`

But, if setting one value of an array:
**To set constraints on just one of the parameters in the x array, we need to add an extra [] to stop the parameter being returned as a simple float!**


In [33]:
r.x[[0]].constrain_bounded(-5,1)
r.x[[1]].constrain_bounded(-2,0.5)

In [34]:

#r.x[1].constrain_bounded(-4.0,1.5)
r.randomize()

In [35]:
r.optimize()

In [36]:
r.x

index,rosen.position,constraints
[0],0.70855950,"-5.0,1.0"
[1],0.49999988,"-2.0,0.5"


Under the hood there is a transformation that's turning the domain of the constraint to the whole of the real numbers. So when we look at `r.optimizer_array` it **won't have the same values as the actual parameters**!

In [37]:
r.optimizer_array

array([  2.97488614,  16.86102108])

To demonstrate we look at the function that is applied to the above;

[TODO I can't quite see how we actually get the **actual** function]

In [40]:
list(r.x.constraints.items())

[(Logistic, array([0])), (Logistic, array([1]))]

In [41]:
list(r.x.constraints.items())[0]

(Logistic, array([0]))

# Meta notes

## Who is now responsible?

Mike is worried that the project will faulter if there isn't a key (paid) go-to person for the project. Also need to decide on direction, etc.

Need cash! 

Possibly from:

 - Amazon
 - EPSRC

## To do

- Need to improve test coverage + unit testing notebooks
- Documentation - multiple ways in, lots of dead old notebooks
- New features
          - tied kernel

## Documentation

The stuff Tania has written;
- adds a menu
- adds a button to download locally
- run on the cloud (azure service completely free)
- tests the notebooks

Key problems currently
- too many entry points
- old code floating about (wrong versions etc)

Idea
- use sklearn's welcome page



### Plan

- slack channel for documentation notebook
- Need an outline
   - notebook A should go there, B there... etc
   - we need to fill this hole here
   - need to document this feature
   - delete that notebook
   
- hunt down and kill all the old notebooks

- Also need to include (in the doc plan/front page):

   - variational methods
   - mean functions
